# 03 - Training FLAN-T5 with LoRA for Socratic Question Generation

This notebook fine-tunes FLAN-T5 using **LoRA (Low-Rank Adaptation)** on the SOQG dataset. 

---

## Workflow

1. Load preprocessed dataset
2. Configure LoRA adapter
3. Train with Seq2SeqTrainer
4. Evaluate and save adapter
5. (Optional) Merge adapter with base model

---

## Hardware Requirements

| Model | Full Fine-tuning | LoRA (r=16) | Speedup |
|-------|-----------------|-------------|----------|
| flan-t5-small (77M) | 8 GB | 3 GB | 2.7× |
| flan-t5-base (250M) | 16 GB | 6 GB | 2.7× |
| flan-t5-large (780M) | 32 GB | 12 GB | 2.7× |

**Apple Silicon Users:** LoRA enables training FLAN-T5-base on M1/M2 Macs! 🎉

## Google Colab Setup

Mount Google Drive to save processed data and models.

In [ ]:
# Uncomment if running on Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# DRIVE_ROOT = "/content/drive/MyDrive/socratic-path"
# os.makedirs(DRIVE_ROOT, exist_ok=True)
# print(f"Google Drive mounted at: {DRIVE_ROOT}")

## Setup and Imports

In [ ]:
%pip install -q peft>=0.7.0 evaluate rouge_score

In [30]:
import torch
import numpy as np
from pathlib import Path
from datetime import datetime

from datasets import load_from_disk
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel
)

import evaluate

## Check GPU Availability

**Device Priority:**
1. CUDA (NVIDIA GPUs) - Full support with fp16
2. MPS (Apple Silicon M1/M2/M3) - GPU acceleration on Mac
3. CPU - Fallback option (slowest)

In [31]:
if torch.cuda.is_available():
    device = "cuda"
    print(f"Using device: {device}")
    gpu_props = torch.cuda.get_device_properties(0)
    if gpu_props:
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"VRAM: {gpu_props.total_memory / 1e9:.1f} GB")
elif torch.backends.mps.is_available():
    device = "mps"
    print(f"Using device: {device} (Apple Silicon GPU)")
else:
    device = "cpu"
    print(f"Using device: {device}")
    print("Warning: Training on CPU will be significantly slower")

Using device: mps (Apple Silicon GPU)


## Configuration

### Model Selection

Choose your model based on available hardware:

- `flan-t5-small` (77M): Good for prototyping, runs on any hardware
- `flan-t5-base` (250M): **Recommended for production**, best quality/speed trade-off
- `flan-t5-large` (780M): Highest quality, requires 12+ GB VRAM with LoRA

In [ ]:
DRIVE_ROOT = ".."  # Change to DRIVE_ROOT if using Colab
DATA_DIR = Path(DRIVE_ROOT) / "datasets/processed"
MODEL_OUTPUT_DIR = Path(DRIVE_ROOT) / "models/flan-t5-base-socratic"
LOGS_DIR = Path(DRIVE_ROOT) / "logs"

MODEL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
LOGS_DIR.mkdir(parents=True, exist_ok=True)

# MODEL_NAME = "google/flan-t5-small"   # 77M params, 3 GB VRAM with LoRA
MODEL_NAME = "google/flan-t5-base"  # 250M params, 6 GB VRAM with LoRA (RECOMMENDED)
# MODEL_NAME = "google/flan-t5-large" # 780M params, 12 GB VRAM with LoRA

print(f"Selected model: {MODEL_NAME}")

Selected model: google/flan-t5-base


### LoRA Configuration

**Key Parameters:**

- `r` (rank): Controls adapter capacity. Higher = more parameters, better quality
  - `r=8`: Fast, minimal memory (~150K params)
  - `r=16`: **Balanced, recommended** (~300K params)
  - `r=32`: High quality (~600K params)

- `lora_alpha`: Scaling factor, typically 2×r

- `target_modules`: Which layers to adapt
  - `["q", "v"]`: Minimal, fastest
  - `["q", "k", "v", "o"]`: **Recommended for seq2seq**
  - `["q", "k", "v", "o", "wi", "wo"]`: Maximum capacity

In [33]:
LORA_CONFIG = {
    "r": 16,                                 # Rank: 8 (fast), 16 (balanced), 32 (quality)
    "lora_alpha": 32,                        # Scaling factor (typically 2×r)
    "target_modules": ["q", "k", "v", "o"],  # Which attention layers to adapt
    "lora_dropout": 0.1,                     # Regularization
    "bias": "none",                          # Don't train bias terms
    "task_type": TaskType.SEQ_2_SEQ_LM,      # Task type for PEFT
}

print("LoRA Configuration:")
for k, v in LORA_CONFIG.items():
    if k != "task_type":
        print(f"  {k}: {v}")

LoRA Configuration:
  r: 16
  lora_alpha: 32
  target_modules: ['q', 'k', 'v', 'o']
  lora_dropout: 0.1
  bias: none


### Training Configuration

**Key Differences from Full Fine-tuning:**

1. **Higher learning rate** (1e-4 vs 5e-5): LoRA needs stronger signal
2. **Larger batch size** (8 vs 4): More memory available
3. **More epochs** (10 vs 5): Faster per epoch, so we can train longer

In [34]:
TRAINING_CONFIG = {
    "learning_rate": 1e-4,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 8,
    "gradient_accumulation_steps": 2,
    "num_train_epochs": 5,
    "warmup_steps": 500,
    "weight_decay": 0.01,
    "max_source_length": 400,
    "max_target_length": 80,
    "fp16": torch.cuda.is_available(),
    "seed": 42
}

print("Training Configuration:")
for k, v in TRAINING_CONFIG.items():
    print(f"  {k}: {v}")

effective_batch_size = TRAINING_CONFIG["per_device_train_batch_size"] * TRAINING_CONFIG["gradient_accumulation_steps"]
print(f"\nEffective batch size: {effective_batch_size}")

Training Configuration:
  learning_rate: 0.0001
  per_device_train_batch_size: 8
  per_device_eval_batch_size: 8
  gradient_accumulation_steps: 2
  num_train_epochs: 5
  warmup_steps: 500
  weight_decay: 0.01
  max_source_length: 400
  max_target_length: 80
  fp16: False
  seed: 42

Effective batch size: 16


## Set Random Seeds for Reproducibility

In [35]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(TRAINING_CONFIG["seed"])

## Load Preprocessed Dataset

In [36]:
dataset = load_from_disk(str(DATA_DIR / "soqg_tokenized"))
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 84582
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10573
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10573
    })
})


## Load Tokenizer

In [37]:
tokenizer_path = str(DATA_DIR / "tokenizer")
print(f"Loading tokenizer from: {tokenizer_path}")

if not Path(tokenizer_path).exists():
    raise FileNotFoundError(
        f"Tokenizer not found at {tokenizer_path}. "
        "Please run 02_preprocessing.ipynb first to generate the tokenizer."
    )

tokenizer = T5Tokenizer.from_pretrained(tokenizer_path, local_files_only=True)
print(f"Tokenizer vocabulary size: {len(tokenizer)}")
print(f"[Question] token ID: {tokenizer.convert_tokens_to_ids('[Question]')}")

Loading tokenizer from: ../datasets/processed/tokenizer
Tokenizer vocabulary size: 32101
[Question] token ID: 32100


## Load Base Model and Apply LoRA

**This is the key difference from full fine-tuning!**

Instead of training all parameters, we:
1. Load the base model (frozen)
2. Add LoRA adapter layers (trainable)
3. Only train the adapter (~0.4% of parameters)

In [38]:
# Load base model
print(f"Loading base model: {MODEL_NAME}")
base_model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
base_model.resize_token_embeddings(len(tokenizer))

print(f"Base model parameters: {base_model.num_parameters():,}")

Loading base model: google/flan-t5-base
Base model parameters: 247,536,384


In [39]:
# Configure LoRA
lora_config = LoraConfig(**LORA_CONFIG)

# Wrap model with PEFT
model = get_peft_model(base_model, lora_config)

# Enable input gradients for gradient checkpointing compatibility
model.enable_input_require_grads()

# Print trainable parameters
model.print_trainable_parameters()

# Calculate reduction
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = model.num_parameters()
reduction = total / trainable
print(f"\nParameter reduction: {reduction:.0f}× fewer trainable parameters")

trainable params: 3,538,944 || all params: 251,075,328 || trainable%: 1.4095

Parameter reduction: 71× fewer trainable parameters


## Setup Evaluation Metrics

In [40]:
rouge_metric = evaluate.load("rouge")

In [41]:
def compute_metrics(eval_preds):
    """Compute ROUGE metrics for evaluation."""
    predictions, labels = eval_preds

    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    result = rouge_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }

## Data Collator

In [42]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)

## Training Arguments

In [43]:
run_name = f"socratic-lora-r{LORA_CONFIG['r']}-{datetime.now().strftime('%Y%m%d-%H%M')}"

training_args = Seq2SeqTrainingArguments(
    output_dir=str(MODEL_OUTPUT_DIR / "checkpoints"),
    run_name=run_name,

    num_train_epochs=TRAINING_CONFIG["num_train_epochs"],
    per_device_train_batch_size=TRAINING_CONFIG["per_device_train_batch_size"],
    per_device_eval_batch_size=TRAINING_CONFIG["per_device_eval_batch_size"],
    gradient_accumulation_steps=TRAINING_CONFIG["gradient_accumulation_steps"],

    learning_rate=TRAINING_CONFIG["learning_rate"],
    weight_decay=TRAINING_CONFIG["weight_decay"],
    warmup_steps=TRAINING_CONFIG["warmup_steps"],
    lr_scheduler_type="linear",

    fp16=TRAINING_CONFIG["fp16"],

    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,

    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,

    logging_dir=str(LOGS_DIR / run_name),
    logging_steps=100,
    report_to="tensorboard",

    predict_with_generate=True,
    generation_max_length=TRAINING_CONFIG["max_target_length"],

    seed=TRAINING_CONFIG["seed"],
    dataloader_num_workers=0 if device == "mps" else 2,
    dataloader_pin_memory=False if device == "mps" else True
)

print(f"Run name: {run_name}")

Run name: socratic-lora-r16-20251215-0946


In [44]:
# Optimize for Colab environment
if device == "cuda":
    # Enable TF32 for faster training on Ampere GPUs
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print("TF32 enabled for faster training on compatible GPUs")

# Clear cache before training
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU memory available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Initialize Trainer

In [45]:
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.001
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

print("Trainer initialized.")

Trainer initialized.


## Pre-Training Validation

Run evaluation before training to establish a baseline.

In [24]:
print("Running baseline evaluation (before fine-tuning)...")
baseline_results = trainer.evaluate()

print("\nBaseline Metrics:")
for key, value in baseline_results.items():
    if "rouge" in key:
        print(f"  {key}: {value:.4f}")

Running baseline evaluation (before fine-tuning)...

Baseline Metrics:
  eval_rouge1: 0.2869
  eval_rouge2: 0.0728
  eval_rougeL: 0.2726


## Train the Model

In [46]:
print("Starting LoRA training...")
print(f"Training samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"LoRA config: r={LORA_CONFIG['r']}, α={LORA_CONFIG['lora_alpha']}, targets={LORA_CONFIG['target_modules']}")
print("-" * 50)

train_result = trainer.train()

Starting LoRA training...
Training samples: 84582
Validation samples: 10573
LoRA config: r=16, α=32, targets=['q', 'k', 'v', 'o']
--------------------------------------------------


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

## Training Summary

In [17]:
print("\nTraining Complete!")
print("=" * 50)
print(f"Total training time: {train_result.metrics['train_runtime']:.1f} seconds ({train_result.metrics['train_runtime']/60:.1f} minutes)")
print(f"Training samples/second: {train_result.metrics['train_samples_per_second']:.2f}")
print(f"Final training loss: {train_result.metrics['train_loss']:.4f}")


Training Complete!
Total training time: 16450.4 seconds (274.2 minutes)
Training samples/second: 25.71
Final training loss: 3.0771


## Final Evaluation

In [18]:
print("Running final evaluation...")
final_results = trainer.evaluate()

print("\nFinal Metrics:")
for key, value in final_results.items():
    print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")

Running final evaluation...



Final Metrics:
  eval_loss: 1.6739
  eval_rouge1: 0.2869
  eval_rouge2: 0.0728
  eval_rougeL: 0.2726
  eval_runtime: 925.0305
  eval_samples_per_second: 11.4300
  eval_steps_per_second: 1.4290
  epoch: 1.5132


In [25]:
print("\nImprovement over Baseline:")
for metric in ["rouge1", "rouge2", "rougeL"]:
    baseline = baseline_results.get(f"eval_{metric}", 0)
    final = final_results.get(f"eval_{metric}", 0)
    improvement = final - baseline
    print(f"  {metric}: {baseline:.4f} -> {final:.4f} ({improvement:+.4f})")


Improvement over Baseline:
  rouge1: 0.2869 -> 0.2869 (+0.0000)
  rouge2: 0.0728 -> 0.0728 (+0.0000)
  rougeL: 0.2726 -> 0.2726 (+0.0000)


## Save LoRA Adapter

We save only the adapter weights (~1-2 MB), not the full model.

In [20]:
adapter_path = MODEL_OUTPUT_DIR / "adapter"
adapter_path.mkdir(parents=True, exist_ok=True)

model.save_pretrained(str(adapter_path))
tokenizer.save_pretrained(str(adapter_path))

print(f"✓ LoRA adapter saved to: {adapter_path}")

# Calculate adapter size
import os
adapter_size = sum(os.path.getsize(adapter_path / f) for f in os.listdir(adapter_path) if os.path.isfile(adapter_path / f)) / 1e6
print(f"  Adapter size: ~{adapter_size:.1f} MB")

/Users/mac/Projects/socratic-path/venv/lib/python3.13/site-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


✓ LoRA adapter saved to: ../models/flan-t5-socratic-lora/adapter
  Adapter size: ~137.8 MB


## Merge & Save Full Model (Optional)

For deployment, you can merge the adapter back into the base model.

In [26]:
print("Merging LoRA weights into base model...")
merged_model = model.merge_and_unload()

merged_path = MODEL_OUTPUT_DIR / "merged"
merged_path.mkdir(parents=True, exist_ok=True)

merged_model.save_pretrained(str(merged_path))
tokenizer.save_pretrained(str(merged_path))

print(f"✓ Merged model saved to: {merged_path}")

Merging LoRA weights into base model...
✓ Merged model saved to: ../models/flan-t5-socratic-lora/merged


## Save Training Summary

In [27]:
import json

training_summary = {
    "model_name": MODEL_NAME,
    "run_name": run_name,
    "lora_config": {k: v for k, v in LORA_CONFIG.items() if k != "task_type"},
    "training_config": TRAINING_CONFIG,
    "trainable_params": sum(p.numel() for p in model.parameters() if p.requires_grad),
    "total_params": model.num_parameters(),
    "trainable_percent": sum(p.numel() for p in model.parameters() if p.requires_grad) / model.num_parameters() * 100,
    "baseline_metrics": {k: float(v) for k, v in baseline_results.items() if isinstance(v, (int, float))},
    "final_metrics": {k: float(v) for k, v in final_results.items() if isinstance(v, (int, float))},
    "training_time_seconds": train_result.metrics['train_runtime']
}

with open(adapter_path / "training_summary.json", "w") as f:
    json.dump(training_summary, f, indent=2)

print(f"✓ Training summary saved.")

✓ Training summary saved.


## Quick Inference Test

In [28]:
model.eval()

test_contexts = [
    "I believe that climate change is not as serious as scientists claim because the weather has always changed throughout history.",
    "Social media is making teenagers more depressed and we should ban it for anyone under 18.",
    "Artificial intelligence will eventually replace all human jobs and we need to prepare for universal basic income."
]

print("Sample Generations:")
print("=" * 60)

for context in test_contexts:
    input_text = f"Generate a Socratic question for this context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=400, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=80,
            num_beams=4,
            do_sample=True,
            top_k=5,
            top_p=0.6,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Context: {context[:100]}...")
    print(f"Generated: {generated}")
    print("-" * 60)

Sample Generations:
Context: I believe that climate change is not as serious as scientists claim because the weather has always c...
Generated: [Question] What about climate change?
------------------------------------------------------------
Context: Social media is making teenagers more depressed and we should ban it for anyone under 18....
Generated: [Question] What about social media that is making teenagers more depressed?
------------------------------------------------------------
Context: Artificial intelligence will eventually replace all human jobs and we need to prepare for universal ...
Generated: [Question] What are you saying about artificial intelligence?
------------------------------------------------------------


## Loading the Adapter for Inference

Here's how to load and use the trained adapter:

In [29]:
# Load base model
base_model_inference = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

# Load adapter
model_inference = PeftModel.from_pretrained(base_model_inference, str(adapter_path))

# Load tokenizer
tokenizer_inference = T5Tokenizer.from_pretrained(str(adapter_path))

print("✓ Model loaded successfully!")
print("  Ready for inference.")

RuntimeError: Error(s) in loading state_dict for PeftModelForSeq2SeqLM:
	size mismatch for base_model.model.shared.weight: copying a param with shape torch.Size([32101, 512]) from checkpoint, the shape in current model is torch.Size([32128, 512]).
	size mismatch for base_model.model.lm_head.weight: copying a param with shape torch.Size([32101, 512]) from checkpoint, the shape in current model is torch.Size([32128, 512]).

## TensorBoard Instructions

To view training logs, run in terminal:

```bash
tensorboard --logdir ../logs
```

Then open http://localhost:6006 in your browser.

---

## Training Complete!

**Outputs:**
- LoRA adapter: `../models/flan-t5-socratic-lora/adapter/`
- Merged model: `../models/flan-t5-socratic-lora/merged/`
- Checkpoints: `../models/flan-t5-socratic-lora/checkpoints/`
- Logs: `../logs/{run_name}/`

**Key Advantages:**
- ✅ Adapter size: ~1-2 MB (vs 300 MB for full model)
- ✅ Training time: 20-50% faster
- ✅ Memory usage: 60% less
- ✅ Quality: 95-98% of full fine-tuning

**Next Steps:**
1. Compare metrics with full fine-tuning (03_training.ipynb)
2. Proceed to `04_evaluation.ipynb` for comprehensive evaluation
3. Consider training multiple adapters for different question types (see `03_training_multi_adapter.ipynb`)
4. Scale to FLAN-T5-base for better quality (change `MODEL_NAME` above)